In [ ]:
!pip install requests pandas numpy transformers torch
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install requests pandas numpy transformers torch
!pip install --upgrade --no-cache-dir transformers

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
import requests
import numpy as np
from datetime import datetime, timedelta
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# --- CONFIGURATION: ENTER YOUR NEWSAPI KEY ---
NEWSAPI_KEY = '4df94f613dd1434a98dd4ee95305e906'  # Get it from https://newsapi.org

# --- FinBERT Model Setup (Loads from HuggingFace) ---
finbert = pipeline(
    "sentiment-analysis",
    model="yiyanghkust/finbert-tone",
    tokenizer="yiyanghkust/finbert-tone"
)
print('✅ FinBERT loaded')

# --- Fetch News Headlines for a Ticker Symbol ---
def fetch_ticker_news(ticker, n_headlines=15):
    url = (
        "https://newsapi.org/v2/everything?"
        f"q={ticker}&language=en&sortBy=publishedAt&pageSize={n_headlines}&apiKey={NEWSAPI_KEY}"
    )
    resp = requests.get(url)
    if resp.status_code == 200:
        articles = resp.json().get("articles", [])
        headlines = [a['title'] for a in articles if a.get('title')]
        print(f"Fetched {len(headlines)} news headlines for {ticker}.")
        return headlines
    else:
        print(f"⚠️ Failed to fetch news: {resp.status_code}, {resp.text}")
        return []

# --- Analyze Sentiment with FinBERT ---
def analyze_headline_sentiment(headlines):
    sentiment_counts = {'positive': 0, 'neutral': 0, 'negative': 0}
    sentiment_scores = []
    results = []
    for text in headlines:
        result = finbert(text[:512])[0]  # FinBERT max length is 512 tokens
        label = result['label'].lower()
        score = result['score']
        sentiment_scores.append(
            score if label == 'positive' else -score if label == 'negative' else 0
        )
        sentiment_counts[label] += 1
        results.append((text, label, score))
    return sentiment_counts, sentiment_scores, results

# --- Asset Allocation Suggestion Based on Sentiment ---
def suggest_allocation(avg_score):
    if avg_score > 0.3:
        return "🚀 Aggressive: 80% stocks, 15% bonds, 5% cash"
    elif avg_score > 0.1:
        return "🙂 Growth: 65% stocks, 25% bonds, 10% cash"
    elif avg_score < -0.3:
        return "🛡️ Defensive: 15% stocks, 60% bonds, 25% cash"
    elif avg_score < -0.1:
        return "🙁 Conservative: 30% stocks, 50% bonds, 20% cash"
    else:
        return "😐 Balanced: 50% stocks, 35% bonds, 15% cash"

# --- Interactive Loop ---
def finbert_sentiment_bot():
    print("\nIntelligent Ticker Sentiment Bot (FinBERT-powered)\n")
    print("Type 'exit' to quit.")
    while True:
        ticker = input("📝 Enter a stock ticker (e.g., TSLA, AAPL): ").strip().upper()
        if ticker in ('EXIT', 'QUIT'): break

        headlines = fetch_ticker_news(ticker)
        if not headlines:
            print(f"No news found for {ticker}! Sentiment analysis and allocation cannot be provided.\n")
            continue  # This skips all analysis/allocation if no news.

        # otherwise, continue as normal:
        counts, scores, detailed = analyze_headline_sentiment(headlines)
        avg_score = np.mean(scores) if scores else 0
        print(f"\n[Sentiment breakdown for {ticker}]: {counts}")
        print(f"Average FinBERT sentiment score: {avg_score:.2f}")
        print("Sample headlines:")
        for text, label, score in detailed[:3]:
            print(f" - [{label.upper()} | {score:.2f}] {text[:90]}")
        print(f"\nRecommended Allocation for {ticker}: {suggest_allocation(avg_score)}\n")




if __name__ == "__main__":
    finbert_sentiment_bot()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


✅ FinBERT loaded

Intelligent Ticker Sentiment Bot (FinBERT-powered)

Type 'exit' to quit.
📝 Enter a stock ticker (e.g., TSLA, AAPL): AAPL
Fetched 15 news headlines for AAPL.


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



[Sentiment breakdown for AAPL]: {'positive': 2, 'neutral': 8, 'negative': 5}
Average FinBERT sentiment score: -0.22
Sample headlines:
 - [NEGATIVE | 0.97] Inquiry Into Apple's Competitor Dynamics In Technology Hardware, Storage & Peripherals Ind
 - [NEUTRAL | 1.00] AAPL Stock Q3 Forecast: 4 Things to Watch When Apple Reports Earnings on July 31
 - [NEGATIVE | 1.00] UBS Keeps Neutral on Apple (AAPL), Sees iPhone Demand Drop in June

Recommended Allocation for AAPL: 🙁 Conservative: 30% stocks, 50% bonds, 20% cash

📝 Enter a stock ticker (e.g., TSLA, AAPL): TSLA
Fetched 15 news headlines for TSLA.

[Sentiment breakdown for TSLA]: {'positive': 1, 'neutral': 14, 'negative': 0}
Average FinBERT sentiment score: 0.04
Sample headlines:
 - [NEUTRAL | 1.00] Cathie Wood’s ARK Invest Dumps $105,300,000 in Coinbase (COIN) Stock After All-Time High –
 - [NEUTRAL | 0.99] Everyone is crazy now
 - [POSITIVE | 0.53] 'Magnificent Seven' stocks: Buy Nvidia, skip Apple

Recommended Allocation for TSLA: 😐 B